In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV, train_test_split
import matplotlib.pyplot as plt
import optuna
import optuna.integration
from optuna.integration import OptunaSearchCV
import optuna.integration.lightgbm as lgb_optuna

In [2]:
def read_data_split_train(file):
    df = pd.read_csv("data.csv", sep=';')
    X, y = df.drop(["ID","TARGET"], axis=1), df["TARGET"]
    X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.1, random_state=8,stratify=y)
    X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.15, random_state=8,stratify=y_temp)
    return X_train, X_val, X_test, y_train, y_val, y_test

In [3]:
X_train, X_val, X_test, y_train, y_val, y_test = read_data_split_train("data.csv")

In [4]:
clf = lgb.LGBMClassifier(random_state=8, importance_type="gain")
folds = KFold(n_splits=5, shuffle=True, random_state=8)

## LightGBM + Optuna (naive)

In [7]:
def objective(trial):
    # define hyperparameters to be tuned
    params = {
        "objective": "binary",
        "metric": "auc",
        "num_leaves": trial.suggest_int("num_leaves", 2, 100),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.1),
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "subsample": trial.suggest_uniform("subsample", 0.1, 1),
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.1, 1),
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-5, 10),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-5, 10),
        "random_state": 42,
        "n_jobs": -1,
    }

    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "auc")

    dtrain = lgb_optuna.Dataset(X_train, y_train)
    dval = lgb_optuna.Dataset(X_val, y_val)

    # define the model
    model = lgb_optuna.train(params,dtrain,valid_sets=[dval],
                             callbacks=[pruning_callback],
                            )

    # evaluate the model
    y_pred = model.predict(X_test)
    auc = roc_auc_score(y_test, y_pred)

    # return the objective value to be minimized
    return 1 - auc

In [ ]:
study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), direction="minimize"
    )
study.optimize(objective, n_trials=10)

## OptunaSearchCV

In [12]:
eval_results={}
opt = OptunaSearchCV(clf, param_distributions={"max_depth": optuna.distributions.IntDistribution(2, 4),
                                               "learning_rate": optuna.distributions.FloatDistribution(high=0.1, log=True, low=0.001),},
                                               n_trials=2, verbose=1, random_state=8, n_jobs=-1, cv=folds, scoring="roc_auc", return_train_score=True)

opt.fit(X_train, y_train,eval_set=[(X_train,y_train),(X_val, y_val)], eval_metric="auc",callbacks=[lgb.early_stopping(10),lgb.log_evaluation(period=10)])

/tmp/ipykernel_9782/632472101.py:2: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  opt = OptunaSearchCV(clf, param_distributions={"max_depth": optuna.distributions.IntDistribution(2, 4),
[I 2023-07-17 01:22:06,191] A new study created in memory with name: no-name-578e0dd5-f67f-42e5-87e5-db70d94ebe0b
[I 2023-07-17 01:22:06,192] Searching the best hyperparameters using 22950 samples...


Training until validation scores don't improve for 10 rounds
[10]	valid_0's auc: 0.723967	valid_0's binary_logloss: 0.510925	valid_1's auc: 0.72428	valid_1's binary_logloss: 0.511209
[10]	valid_0's auc: 0.761013	valid_0's binary_logloss: 0.463964	valid_1's auc: 0.751909	valid_1's binary_logloss: 0.465758
[20]	valid_0's auc: 0.759686	valid_0's binary_logloss: 0.497595	valid_1's auc: 0.751923	valid_1's binary_logloss: 0.498058
[20]	valid_0's auc: 0.768284	valid_0's binary_logloss: 0.446395	valid_1's auc: 0.758571	valid_1's binary_logloss: 0.44903
[30]	valid_0's auc: 0.75812	valid_0's binary_logloss: 0.48726	valid_1's auc: 0.751056	valid_1's binary_logloss: 0.487968
[30]	valid_0's auc: 0.772833	valid_0's binary_logloss: 0.438643	valid_1's auc: 0.763319	valid_1's binary_logloss: 0.441585
[40]	valid_0's auc: 0.761098	valid_0's binary_logloss: 0.479031	valid_1's auc: 0.751509	valid_1's binary_logloss: 0.480065
[40]	valid_0's auc: 0.774956	valid_0's binary_logloss: 0.434411	valid_1's auc: 0.7

[I 2023-07-17 01:22:08,649] Trial 1 finished with value: 0.7617711310760507 and parameters: {'max_depth': 2, 'learning_rate': 0.010822912485045274}. Best is trial 1 with value: 0.7617711310760507.


[70]	valid_0's auc: 0.761024	valid_0's binary_logloss: 0.462299	valid_1's auc: 0.751805	valid_1's binary_logloss: 0.464099
[80]	valid_0's auc: 0.762489	valid_0's binary_logloss: 0.458566	valid_1's auc: 0.752179	valid_1's binary_logloss: 0.460609
[60]	valid_0's auc: 0.780142	valid_0's binary_logloss: 0.430508	valid_1's auc: 0.771529	valid_1's binary_logloss: 0.434212
[90]	valid_0's auc: 0.7633	valid_0's binary_logloss: 0.455412	valid_1's auc: 0.753518	valid_1's binary_logloss: 0.457686
[70]	valid_0's auc: 0.782148	valid_0's binary_logloss: 0.429142	valid_1's auc: 0.773488	valid_1's binary_logloss: 0.43302
Early stopping, best iteration is:
[75]	valid_0's auc: 0.783292	valid_0's binary_logloss: 0.428537	valid_1's auc: 0.775271	valid_1's binary_logloss: 0.432347
[80]	valid_0's auc: 0.784374	valid_0's binary_logloss: 0.427841	valid_1's auc: 0.77612	valid_1's binary_logloss: 0.431817
[90]	valid_0's auc: 0.785849	valid_0's binary_logloss: 0.426843	valid_1's auc: 0.77702	valid_1's binary_logl

[I 2023-07-17 01:22:09,141] Trial 0 finished with value: 0.7779272654323757 and parameters: {'max_depth': 2, 'learning_rate': 0.06908179863517087}. Best is trial 0 with value: 0.7779272654323757.
[I 2023-07-17 01:22:09,142] Finished hyperparemeter search!
[I 2023-07-17 01:22:09,143] Refitting the estimator using 22950 samples...


[50]	valid_0's auc: 0.778213	valid_0's binary_logloss: 0.431683	valid_1's auc: 0.769206	valid_1's binary_logloss: 0.436209
[60]	valid_0's auc: 0.780018	valid_0's binary_logloss: 0.429909	valid_1's auc: 0.771289	valid_1's binary_logloss: 0.434566
[70]	valid_0's auc: 0.781148	valid_0's binary_logloss: 0.428671	valid_1's auc: 0.772039	valid_1's binary_logloss: 0.43357
[80]	valid_0's auc: 0.782696	valid_0's binary_logloss: 0.427596	valid_1's auc: 0.773173	valid_1's binary_logloss: 0.432621
[90]	valid_0's auc: 0.784357	valid_0's binary_logloss: 0.426593	valid_1's auc: 0.774926	valid_1's binary_logloss: 0.431622
[100]	valid_0's auc: 0.78575	valid_0's binary_logloss: 0.42566	valid_1's auc: 0.776284	valid_1's binary_logloss: 0.430721
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.786916	valid_0's binary_logloss: 0.42596	valid_1's auc: 0.777746	valid_1's binary_logloss: 0.430576
[10]	training's auc: 0.75941	training's binary_logloss: 0.463981	valid_1's auc: 0.752397	vali

[I 2023-07-17 01:22:09,438] Finished refitting! (elapsed time: 0.294 sec.)


[100]	training's auc: 0.786582	training's binary_logloss: 0.425477	valid_1's auc: 0.774874	valid_1's binary_logloss: 0.431061
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.786916	valid_0's binary_logloss: 0.42596	valid_1's auc: 0.777746	valid_1's binary_logloss: 0.430576


OptunaSearchCV(cv=KFold(n_splits=5, random_state=8, shuffle=True),
               estimator=LGBMClassifier(importance_type='gain', random_state=8),
               n_jobs=-1, n_trials=2,
               param_distributions={'learning_rate': FloatDistribution(high=0.1, log=True, low=0.001, step=None),
                                    'max_depth': IntDistribution(high=4, log=False, low=2, step=1)},
               random_state=8, return_train_score=True, scoring='roc_auc',
               verbose=1)

## LightGBMTuner

In [24]:
# define hyperparameters to be tuned
params = {"objective": "binary",
        "metric": "binary_logloss",
        # "verbosity": 1,
        "boosting_type": "gbdt",
        "n_jobs": -1
}
dtrain = lgb_optuna.Dataset(X_train, y_train)
dval = lgb_optuna.Dataset(X_val, y_val)

In [25]:
model = lgb_optuna.train(params, dtrain, valid_sets=[dtrain, dval], 
                         callbacks=[lgb.early_stopping(3),lgb.log_evaluation(period=10)])

[I 2023-07-17 02:03:41,913] A new study created in memory with name: no-name-030cb193-949a-4ce0-8e35-4870be4c0be6
feature_fraction, val_score: 0.425302:  14%|#4        | 1/7 [00:00<00:01,  4.56it/s]

[LightGBM] [Info] Number of positive: 5076, number of negative: 17874
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003023 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3265
[LightGBM] [Info] Number of data points in the train set: 22950, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.221176 -> initscore=-1.258824
[LightGBM] [Info] Start training from score -1.258824
Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.443873	valid_1's binary_logloss: 0.449979
[20]	valid_0's binary_logloss: 0.418129	valid_1's binary_logloss: 0.431118
[30]	valid_0's binary_logloss: 0.404493	valid_1's binary_logloss: 0.425965
[40]	valid_0's binary_logloss: 0.395246	valid_1's binary_logloss: 0.425422
Early stopping, best iteration is:
[38]	valid_0's binary_logloss: 0.396877	valid_1's binary_logloss: 0.425302
[LightGBM] [Info] Number of posit

feature_fraction, val_score: 0.425302:  29%|##8       | 2/7 [00:00<00:01,  3.48it/s]

[40]	valid_0's binary_logloss: 0.392585	valid_1's binary_logloss: 0.426052
Early stopping, best iteration is:
[38]	valid_0's binary_logloss: 0.396877	valid_1's binary_logloss: 0.425302
[LightGBM] [Info] Number of positive: 5076, number of negative: 17874
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000389 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3265
[LightGBM] [Info] Number of data points in the train set: 22950, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.221176 -> initscore=-1.258824
[LightGBM] [Info] Start training from score -1.258824
[10]	valid_0's binary_logloss: 0.453317	valid_1's binary_logloss: 0.459751
[20]	valid_0's binary_logloss: 0.424176	valid_1's binary_logloss: 0.437212
[30]	valid_0's binary_logloss: 0.409987	valid_1's binary_logloss: 0.429731


feature_fraction, val_score: 0.425302:  43%|####2     | 3/7 [00:00<00:01,  3.98it/s]

...

feature_fraction_stage2, val_score: 0.424381:  67%|######6   | 4/6 [00:01<00:00,  3.04it/s]

[60]	valid_0's binary_logloss: 0.379628	valid_1's binary_logloss: 0.425464
Early stopping, best iteration is:
[53]	valid_0's binary_logloss: 0.219745	valid_1's binary_logloss: 0.440925
[LightGBM] [Info] Number of positive: 5076, number of negative: 17874
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000769 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3265
[LightGBM] [Info] Number of data points in the train set: 22950, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.221176 -> initscore=-1.258824
[LightGBM] [Info] Start training from score -1.258824
[10]	valid_0's binary_logloss: 0.441231	valid_1's binary_logloss: 0.447832
[20]	valid_0's binary_logloss: 0.416449	valid_1's binary_logloss: 0.4302
[30]	valid_0's binary_logloss: 0.403423	valid_1's binary_logloss: 0.426256


[W 2023-07-17 02:03:58,853] Trial 41 failed with parameters: {'feature_fraction': 0.784} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/workspaces/ml_utils/venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/workspaces/ml_utils/venv/lib/python3.10/site-packages/optuna/integration/_lightgbm_tuner/optimize.py", line 240, in __call__
    booster = lgb.train(self.lgbm_params, train_set, **kwargs)
  File "/workspaces/ml_utils/venv/lib/python3.10/site-packages/lightgbm/engine.py", line 292, in train
    booster.update(fobj=fobj)
  File "/workspaces/ml_utils/venv/lib/python3.10/site-packages/lightgbm/basic.py", line 3021, in update
    _safe_call(_LIB.LGBM_BoosterUpdateOneIter(
KeyboardInterrupt
[W 2023-07-17 02:03:58,854] Trial 41 failed with value None.


[40]	valid_0's binary_logloss: 0.394397	valid_1's binary_logloss: 0.424498


KeyboardInterrupt: 

## LightGBMTunerCV

In [22]:
# define hyperparameters to be tuned
params = {"objective": "binary",
        "metric": "binary_logloss",
        # "verbosity": 1,
        "boosting_type": "gbdt",
        "n_jobs": -1
}

dtrain = lgb_optuna.Dataset(X_train, y_train)
dval = lgb_optuna.Dataset(X_val, y_val)

folds = KFold(n_splits=5, shuffle=True, random_state=8)
tuner = lgb_optuna.LightGBMTunerCV(params=params,train_set=dtrain,folds=folds,num_boost_round=10,
                                    seed=8,optuna_seed=8,time_budget=60,
                                    callbacks=[lgb.early_stopping(3),lgb.log_evaluation(100)])
tuner.run()

[I 2023-07-17 02:00:11,015] A new study created in memory with name: no-name-260cf369-13cb-49b5-95c2-abaa637b191e
feature_fraction, val_score: 0.452732:  14%|#4        | 1/7 [00:00<00:01,  4.42it/s]

[LightGBM] [Info] Number of positive: 4070, number of negative: 14290
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000487 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3265
[LightGBM] [Info] Number of data points in the train set: 18360, number of used features: 23
[LightGBM] [Info] Number of positive: 4093, number of negative: 14267
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000475 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3265
[LightGBM] [Info] Number of data points in the train set: 18360, number of used features: 23
[LightGBM] [Info] Number of positive: 4047, number of negative: 14313
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of t

feature_fraction, val_score: 0.443360:  29%|##8       | 2/7 [00:00<00:01,  4.45it/s]

[LightGBM] [Info] Number of positive: 4070, number of negative: 14290
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000724 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3265
[LightGBM] [Info] Number of data points in the train set: 18360, number of used features: 23
[LightGBM] [Info] Number of positive: 4093, number of negative: 14267
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000711 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3265
[LightGBM] [Info] Number of data points in the train set: 18360, number of used features: 23
[LightGBM] [Info] Number of positive: 4047, number of negative: 14313
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of t

feature_fraction, val_score: 0.443360:  43%|####2     | 3/7 [00:00<00:00,  4.75it/s]

...

Did not meet early stopping. Best iteration is:
[10]	cv_agg's binary_logloss: 0.441598 + 0.00576418
[LightGBM] [Info] Number of positive: 4070, number of negative: 14290
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004222 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3265
[LightGBM] [Info] Number of data points in the train set: 18360, number of used features: 23
[LightGBM] [Info] Number of positive: 4093, number of negative: 14267
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000708 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3265
[LightGBM] [Info] Number of data points in the train set: 18360, number of used features: 23
[LightGBM] [Info] Number of positive: 4047, number of negative: 14313
[LightGBM] [Warning] Auto-choosing row-wis

min_data_in_leaf, val_score: 0.441598:  80%|########  | 4/5 [00:02<00:00,  1.66it/s]

Did not meet early stopping. Best iteration is:
[10]	cv_agg's binary_logloss: 0.441598 + 0.00576418
[LightGBM] [Info] Number of positive: 4070, number of negative: 14290
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000687 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3265
[LightGBM] [Info] Number of data points in the train set: 18360, number of used features: 23
[LightGBM] [Info] Number of positive: 4093, number of negative: 14267
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000691 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3265
[LightGBM] [Info] Number of data points in the train set: 18360, number of used features: 23
[LightGBM] [Info] Number of positive: 4047, numb

min_data_in_leaf, val_score: 0.441598: 100%|##########| 5/5 [00:02<00:00,  1.70it/s]

Did not meet early stopping. Best iteration is:
[10]	cv_agg's binary_logloss: 0.441598 + 0.00576418


In [23]:
tuner.best_params

{'objective': 'binary',
 'metric': 'binary_logloss',
 'boosting_type': 'gbdt',
 'n_jobs': -1,
 'feature_pre_filter': False,
 'lambda_l1': 1.4299183901170552e-06,
 'lambda_l2': 3.3642476428392784e-07,
 'num_leaves': 164,
 'feature_fraction': 0.9840000000000001,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 100}

## Custom LGBM+Optuna

In [9]:
from sklearn.model_selection import cross_validate

In [10]:
def custom_metric(train,valid):
    alpha = 3
    beta = 0.1
    return alpha*valid - beta*abs(train-valid)

In [26]:
def objective(trial):
    # define hyperparameters to be tuned
    params = {
        "num_leaves": trial.suggest_int("num_leaves", 2, 100),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.1),
        "n_estimators": 1000,
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "subsample": trial.suggest_uniform("subsample", 0.1, 1),
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.1, 1),
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-5, 10),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-5, 10),
        "random_state": 42,
        "n_jobs": -1,

    }

    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "auc")

    # define the model
    model = lgb.LGBMClassifier(**params)

    # train the model
    # model.fit(X_train, y_train,
    #            callbacks=[lgb.early_stopping(10),lgb.log_evaluation(100),pruning_callback])
    cross_results = cross_validate(model, X_train, y_train, cv=5, scoring="roc_auc", n_jobs=-1,
                                   return_train_score=True,
                                   fit_params={"eval_set":[(X_train,y_train),(X_val, y_val)],
                                               "eval_metric":"auc",
                                               "callbacks": [lgb.early_stopping(10),lgb.log_evaluation(100),pruning_callback]})

    # evaluate the model
    result = custom_metric(cross_results["train_score"].mean(), cross_results["test_score"].mean())

    # return the objective value to be maximized
    return result

In [27]:
study = optuna.create_study(direction="maximize",
                            pruner=optuna.pruners.MedianPruner(n_warmup_steps=10))
study.optimize(objective, n_trials=10)

[I 2023-07-17 02:43:42,346] A new study created in memory with name: no-name-336bfa47-eafe-447f-ab17-48302c284c09
/tmp/ipykernel_34418/1012821685.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.1),
/tmp/ipykernel_34418/1012821685.py:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "subsample": trial.suggest_uniform("subsample", 0.1, 1),
/tmp/ipykernel_34418/1012821685.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "colsample_bytree": trial.suggest_uniform("colsample_bytre

Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[8]	valid_0's auc: 0.808465	valid_0's binary_logloss: 0.51729	valid_1's auc: 0.786613	valid_1's binary_logloss: 0.518026
Early stopping, best iteration is:
[6]	valid_0's auc: 0.810995	valid_0's binary_logloss: 0.519705	valid_1's auc: 0.786687	valid_1's binary_logloss: 0.520352
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[42]	valid_0's auc: 0.815413	valid_0's binary_logloss: 0.482269	valid_1's auc: 0.782822	valid_1's binary_logloss: 0.486261
Early stopping, best iteration is:
[39]	valid_0's auc: 0.814871	valid_0's binary_logloss: 0.48435	valid_1's auc: 0.784532	valid_1's binary_logloss: 0.488695
Early stopping, best iteration is:
[39]	valid_0's auc: 0.815908	val

[I 2023-07-17 02:43:44,297] Trial 0 finished with value: 2.334146727384563 and parameters: {'num_leaves': 76, 'learning_rate': 0.006517726760599603, 'min_child_samples': 71, 'subsample': 0.34500310254184346, 'colsample_bytree': 0.8459297633481696, 'reg_alpha': 0.21108797152737233, 'reg_lambda': 0.0005712424191967936}. Best is trial 0 with value: 2.334146727384563.
/tmp/ipykernel_34418/1012821685.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.1),
/tmp/ipykernel_34418/1012821685.py:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "subsample": trial.suggest_uniform("subsample", 0.1, 1),
/tmp/ipykernel_34418/1012821685.

Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[8]	valid_0's auc: 0.798654	valid_0's binary_logloss: 0.512658	valid_1's auc: 0.780598	valid_1's binary_logloss: 0.513796
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[21]	valid_0's auc: 0.803404	valid_0's binary_logloss: 0.492165	valid_1's auc: 0.780416	valid_1's binary_logloss: 0.495
Early stopping, best iteration is:
[50]	valid_0's auc: 0.808691	valid_0's binary_logloss: 0.460697	valid_1's auc: 0.783401	valid_1's binary_logloss: 0.467858
Early stopping, best iteration is:
[50]	valid_0's auc: 0.809104	valid_0's binary_logloss: 0.461137	valid_1's auc: 0.783192	valid_1's binary_logloss: 0.46735
Early stopping, best iteration is:
[21]	valid_0's auc: 0.804389	vali

[I 2023-07-17 02:43:45,621] Trial 1 finished with value: 2.3355376215412553 and parameters: {'num_leaves': 52, 'learning_rate': 0.010922604884572877, 'min_child_samples': 16, 'subsample': 0.21720552601343146, 'colsample_bytree': 0.5141748576189217, 'reg_alpha': 0.0010408459185322007, 'reg_lambda': 0.004639817684315513}. Best is trial 1 with value: 2.3355376215412553.
/tmp/ipykernel_34418/1012821685.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.1),
/tmp/ipykernel_34418/1012821685.py:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "subsample": trial.suggest_uniform("subsample", 0.1, 1),
/tmp/ipykernel_34418/10128216

...

[I 2023-07-17 02:43:53,470] Trial 8 finished with value: 2.3325438715797966 and parameters: {'num_leaves': 89, 'learning_rate': 0.008423247483704092, 'min_child_samples': 8, 'subsample': 0.9747751715882267, 'colsample_bytree': 0.7999342455855488, 'reg_alpha': 0.18099973462968197, 'reg_lambda': 1.3476104897239582}. Best is trial 1 with value: 2.3355376215412553.


Early stopping, best iteration is:
[35]	valid_0's auc: 0.823444	valid_0's binary_logloss: 0.479283	valid_1's auc: 0.782176	valid_1's binary_logloss: 0.485626


/tmp/ipykernel_34418/1012821685.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.1),
/tmp/ipykernel_34418/1012821685.py:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "subsample": trial.suggest_uniform("subsample", 0.1, 1),
/tmp/ipykernel_34418/1012821685.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.1, 1),
/tmp/ipykernel_34418/1012821685.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.

Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[12]	valid_0's auc: 0.812465	valid_0's binary_logloss: 0.489545	valid_1's auc: 0.77603	valid_1's binary_logloss: 0.494235
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[34]	valid_0's auc: 0.830249	valid_0's binary_logloss: 0.443747	valid_1's auc: 0.780924	valid_1's binary_logloss: 0.458235
Early stopping, best iteration is:
[34]	valid_0's auc: 0.831536	valid_0's binary_logloss: 0.444461	valid_1's auc: 0.779195	valid_1's binary_logloss: 0.459714
Early stopping, best iteration is:
[87]	valid_0's auc: 0.855598	valid_0's binary_logloss: 0.396255	valid_1's auc: 0.785556	valid_1's binary_logloss: 0.432262
[100]	valid_0's auc: 0.860892	valid_0's binary_logloss: 0.389069

[I 2023-07-17 02:43:55,260] Trial 9 finished with value: 2.321250103211864 and parameters: {'num_leaves': 96, 'learning_rate': 0.03137815008200072, 'min_child_samples': 44, 'subsample': 0.583273509001468, 'colsample_bytree': 0.20807940169870992, 'reg_alpha': 0.00018003298352337068, 'reg_lambda': 2.0885911295593627e-05}. Best is trial 1 with value: 2.3355376215412553.


In [28]:
study.best_params

{'num_leaves': 52,
 'learning_rate': 0.010922604884572877,
 'min_child_samples': 16,
 'subsample': 0.21720552601343146,
 'colsample_bytree': 0.5141748576189217,
 'reg_alpha': 0.0010408459185322007,
 'reg_lambda': 0.004639817684315513}

In [29]:
study.best_value

2.3355376215412553